In [1]:
# !wget https://ml-coding-test.s3.eu-west-1.amazonaws.com/webis_train.csv
# !wget https://ml-coding-test.s3.eu-west-1.amazonaws.com/webis_test.csv
     

In [2]:
import pandas as pd

train = pd.read_csv("webis_train.csv", usecols=["postText", "truthClass"])
test = pd.read_csv("webis_test.csv", usecols=["postText", "truthClass"])

train.rename(columns={"postText": "text", "truthClass": "label"}, inplace=True)
test.rename(columns={"postText": "text", "truthClass": "label"}, inplace=True)

In [3]:
train.shape, test.shape

((19538, 2), (18979, 2))

In [4]:
train.isna().sum(), test.isna().sum()

(text     54
 label     0
 dtype: int64,
 text     66
 label     0
 dtype: int64)

In [5]:
train = train.dropna(subset=["text"]).reset_index(drop=True)
test = test.dropna(subset=["text"]).reset_index(drop=True)

In [6]:
import torch
import random
import numpy as np

# Set a fixed seed value for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [7]:
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer
from datasets import Dataset, DatasetDict, load_dataset, concatenate_datasets, ClassLabel
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

In [8]:
dataset = concatenate_datasets(
    [
        Dataset.from_pandas(train, split="train"),
        Dataset.from_pandas(test, split="test"),
    ]
)

dataset = dataset.cast_column("label", ClassLabel(names=["no-clickbait", "clickbait"]))

Casting the dataset:   0%|          | 0/38397 [00:00<?, ? examples/s]

In [9]:
# SAMPLE_SIZE = 15000

# dataset = dataset.shuffle(seed=SEED).select([i for i in list(range(SAMPLE_SIZE))])

train_test = dataset.train_test_split(test_size=0.3, stratify_by_column="label")
eval_test = train_test["test"].train_test_split(test_size=0.5)

webis17 = DatasetDict(
    {
        "train": train_test["train"],
        "eval": eval_test["train"],
        "test": eval_test["test"],
    }
)

webis17

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 26877
    })
    eval: Dataset({
        features: ['text', 'label'],
        num_rows: 5760
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 5760
    })
})

In [10]:
import torch
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
)
from datasets import load_metric

MODEL_NAME = "distilbert/distilbert-base-uncased"


# Move the model to the GPU (if available)
device = "mps" if torch.backends.mps.is_available() else "cpu"
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


# Define label mappings
num_classes = len(webis17["train"].features["label"].names)
id2label = {id: webis17["train"].features["label"].int2str(id) for id in range(num_classes)}
label2id = {label: id for (id, label) in id2label.items()}


# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=num_classes,
    device_map=device,
    id2label=id2label,
    label2id=label2id
)


# Tokenize the datasets
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=model.config.max_position_embeddings,
    )


tokenized_datasets = webis17.map(tokenize_function, batched=True)

# Load metric
metric = load_metric("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


# Define training arguments
training_args = TrainingArguments(
    output_dir="./checkpoints",
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["eval"],
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

# Save the fine-tuned model
model.save_pretrained(f"{MODEL_NAME.split('/')[1]}_webis17_tuned")
tokenizer.save_pretrained(f"{MODEL_NAME.split('/')[1]}_webis17_tuned")

# Test the model
test_results = trainer.predict(tokenized_datasets["test"])
print(f"Test results: {test_results.metrics}")

Using device: mps


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/26877 [00:00<?, ? examples/s]

Map:   0%|          | 0/5760 [00:00<?, ? examples/s]

Map:   0%|          | 0/5760 [00:00<?, ? examples/s]

...

{'eval_loss': 0.4440048336982727, 'eval_accuracy': 0.8607638888888889, 'eval_runtime': 96.2392, 'eval_samples_per_second': 59.851, 'eval_steps_per_second': 0.935, 'epoch': 3.0}
{'train_runtime': 4350.0188, 'train_samples_per_second': 18.536, 'train_steps_per_second': 0.579, 'train_loss': 0.2550725601968311, 'epoch': 3.0}


  0%|          | 0/90 [00:00<?, ?it/s]

Evaluation results: {'eval_loss': 0.33016437292099, 'eval_accuracy': 0.85625, 'eval_runtime': 106.7996, 'eval_samples_per_second': 53.933, 'eval_steps_per_second': 0.843, 'epoch': 3.0}


  0%|          | 0/90 [00:00<?, ?it/s]

Test results: {'test_loss': 0.3227173388004303, 'test_accuracy': 0.8526041666666667, 'test_runtime': 109.7356, 'test_samples_per_second': 52.49, 'test_steps_per_second': 0.82}


In [11]:
# With both the model and tokenizer initialized we are now able to get explanations on an example text.

from transformers_interpret import SequenceClassificationExplainer

cls_explainer = SequenceClassificationExplainer(model.to("cpu"), tokenizer)

In [12]:
word_attributions = cls_explainer(
    "Shocking Revelation: The Secret Ingredient That Could Change Your Life Forever!",
    class_name="clickbait",
)
word_attributions

[('[CLS]', 0.0),
 ('shocking', 0.11747963820298502),
 ('revelation', -0.020785073184324515),
 (':', 0.3084187505542698),
 ('the', 0.43380586372039176),
 ('secret', 0.4525724881453869),
 ('ingredient', 0.29264621996251317),
 ('that', 0.11694752922367531),
 ('could', 0.07316515925753167),
 ('change', -0.32647736980122866),
 ('your', 0.3040690585780369),
 ('life', -0.17935635293153954),
 ('forever', -0.03716371129115278),
 ('!', 0.40045976313846154),
 ('[SEP]', 0.0)]

In [13]:
cls_explainer.predicted_class_index, cls_explainer.predicted_class_name

(array(1), 'clickbait')

In [14]:
cls_explainer.visualize("viz.html", true_class="clickbait");

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
clickbait,clickbait (0.95),clickbait,1.94,[CLS] shocking revelation : the secret ingredient that could change your life forever ! [SEP]


In [15]:
id2label, label2id

({0: 'no-clickbait', 1: 'clickbait'}, {'no-clickbait': 0, 'clickbait': 1})

### Building vocabulary with attribution score

In [30]:
corpus = dataset.to_pandas()
corpus["label"] = corpus.label.map({0: "no-clickbait", 1: "clickbait"})

In [42]:
from tqdm import tqdm

attribution_all = []

# Get the total number of rows to be processed
total_rows = len(corpus.query("label == 'clickbait'"))

# Wrap the iterable with tqdm for the progress bar
for i, row in tqdm(corpus.query("label == 'clickbait'").iterrows(), total=total_rows):
    attribution_all.append(
        cls_explainer(corpus.loc[i, "text"], class_name=corpus.loc[i, "label"])
    )

100%|██████████| 9174/9174 [43:31<00:00,  3.51it/s]


In [43]:
import pickle

# Specify the filename for the pickle file
filename = "vocabulary_clickbait_attribution.pkl"

# Open the file in write-binary mode ('wb')
with open(filename, "wb") as file:
    # Serialize and save the attribution_all object to the file
    pickle.dump(attribution_all, file)

In [ ]:
# # Open the file in read-binary mode ('rb')
# with open(filename, "rb") as file:
#     # Deserialize the object from the file
#     vocabulary_attribution = pickle.load(file)